## Project Milestone 4 - Linguistic Harbingers of Betrayal Extension

First, input all the libraries we need to get the plot:

In [8]:
import json

from scipy.stats import sem
import numpy as np
import matplotlib.pyplot as plt

Since the data file contains only the selected games, there is no need to consider the selection criteria when retreving the message data. 
*The condition "relationships that contain at least two consecutive and reciprocated acts of support that span at least three seasons in game time, with no more than five seasons passing between two acts of friendship" is fullfilled.

Here, we read the data from json file and get the number of games, which is 500, including 250 selected games ended up betrayal and 250 selected games ended up lasting friendship between two players.


In [9]:
# read data from diplomacy_data.json with reading mode
with open("diplomacy_data.json", "r") as f:
        data = json.load(f)
len(data)

500

Since we only consider dialogs that happen before the last act of friendship, we need to know when does the last act of friendship happen. Later when extracting feature values, extract them from the seasons that is before the last support season.

In [10]:
# a function to get the season where the last support between two players happened
# entry refer to one entry in the dataset, one game
def last_support(entry):
    seasons = entry['seasons']
    last_support = None
    for season in seasons[:-1]:
        if 'support' in season['interaction'].values():
            last_support = season['season']
    return last_support

Given the messages sent by a player from a season in a game, we need to calculate the different variable values of the messages.

Variables we consider:

1. sent_pos: Positive sentiment
2. sent_neu: Neutral sentiment
3. sent_neg: Negative sentiment
4. discourse_comp: Discourse complexity (calculated through comparison, contingency, expansive and temporal)
5. plan: Planning level (calculated through future)
6. argu_level: Argumentation level (calculated through claim and premise)
7. n_request: Number of requests
8. politeness: Politeness
9. subj: Subjectivity
10. n_words: Number of words
11. n_sentences: Number of sentences


In [16]:
# a function to get the average value across the seasons for all features of the messages
# msgs: messages sent by a player from a season in a game
def extract_features(msgs):
    n_sents = sum(m['n_sentences'] for m in msgs) * 1.0
    
    # Sentiment
    # compute positive sentiment score
    sent_pos = sum(m['sentiment'].get("positive") for m in msgs) / len(msgs)
    # compute neutral sentiment score
    sent_neu = sum(m['sentiment'].get("neutral") for m in msgs) / len(msgs)
    # compute positive sentiment score
    sent_neg = sum(m['sentiment'].get("negative") for m in msgs) / len(msgs)
    
    # Argumentation and Discourse
    # compute discourse complexity through comparison, contingency, expansive and temporal
    comparison = sum(len(m['lexicon_words'].get("disc_comparison", []))for m in msgs) / n_sents
    contingency = sum(len(m['lexicon_words'].get("disc_contingency", []))for m in msgs) / n_sents
    expansive = sum(len(m['lexicon_words'].get("disc_expansion", []))for m in msgs) / n_sents
    temporal = sum(len(m['lexicon_words'].get("disc_temporal_rest", []))for m in msgs) / n_sents
    discourse_comp = comparison+contingency+expansive+temporal
    # compute the average number of markers refer to future
    plan = sum(len(m['lexicon_words'].get("disc_temporal_future", []))for m in msgs) / n_sents
    # compute argumentation level through claim and premise
    claim = sum(len(m['lexicon_words'].get("claim", []))for m in msgs) / n_sents
    premise = sum(len(m['lexicon_words'].get("premise", []))for m in msgs) / n_sents
    argu_level = claim+premise
    # compute average number of request
    n_request = sum(m['n_requests'] for m in msgs) / len(msgs)
    
    # Politeness
    politeness = sum(m['politeness'] for m in msgs) / len(msgs)
    
    # Subjectivity using allsubj
    subj = sum(len(m['lexicon_words'].get("allsubj", []))for m in msgs) / n_sents
    
    # Talkativeness
    # compute average number of words
    n_words = sum(m['n_words'] for m in msgs) / len(msgs)
    # compute average number of sentences
    n_sentences = sum(m['n_sentences'] for m in msgs) / len(msgs)

    return dict(
        sentPos = sent_pos,
        sentNeu = sent_neu,
        sentNeg = sent_neg,
        discourseComp = discourse_comp,
        plan = plan,
        arguLevel = argu_level,
        nRequest = n_request,
        politeness = politeness,
        subjectivity = subj,
        nWords = n_words,
        nSentences = n_sentences)

From paper: To ensure that we are studying conversational patterns that occur only when the two individuals in the dyad are ostensibly being friends, we only extract features from the messages exchanged before the last act of friendship.
Therefore, the messages we extract should from seasons less than last support season.

In [17]:
# a function to get average politeness scores from dataset entries 
# data: the dataset where we extract politeness score from
# betrayal: if the games we consider end up betrayal
# betrayer: if the person to be analyzed is the (potential) betrayer
def process_data(data, betrayal, betrayer):
    results = []
    
    # loop in every game in data
    for entry in data:
        for season in entry['seasons']:
            # consider every season in the game, extract features from the messages exchanged before the last act of friendship
            if(season['season'] <= last_support(entry)):
                if(len(season['messages']['betrayer']) > 0 and len(season['messages']['victim']) > 0):
                    if(entry['betrayal'] == betrayal):
                        if(betrayer):
                            results.append(extract_features(season['messages']['betrayer']))
                        else:
                            results.append(extract_features(season['messages']['victim']))
    # return value is a list of features of the indicated player in the indicated type of games
    return results

Get the politeness score of the four conditions below:

In [18]:
# for the games end up betrayal, get politeness score for each betrayer
betray_er = process_data(data, True, True)

# for the games end up betrayal, get politeness score for each victim
betray_vi = process_data(data, True, False) 

# for the games not end up betrayal, get politeness score for each potential betrayer
control_er = process_data(data, False, True) 

# for the games not end up betrayal, get politeness score for each potential victim
control_vi = process_data(data, False, False) 

In [19]:
betray_er

[{'sentPos': 1.3333333333333333,
  'sentNeu': 1.3333333333333333,
  'sentNeg': 1.5,
  'discourseComp': 0.68,
  'plan': 0.12,
  'arguLevel': 0.32,
  'nRequest': 3.6666666666666665,
  'politeness': 0.8033280896385558,
  'subjectivity': 3.68,
  'nWords': 81.5,
  'nSentences': 4.166666666666667},
 {'sentPos': 0.14285714285714285,
  'sentNeu': 0.8571428571428571,
  'sentNeg': 1.2857142857142858,
  'discourseComp': 0.375,
  'plan': 0.125,
  'arguLevel': 0.25,
  'nRequest': 1.2857142857142858,
  'politeness': 0.560083068172233,
  'subjectivity': 2.125,
  'nWords': 40.0,
  'nSentences': 2.2857142857142856},
 {'sentPos': 2.0,
  'sentNeu': 2.5,
  'sentNeg': 2.0,
  'discourseComp': 1.5384615384615385,
  'plan': 0.3076923076923077,
  'arguLevel': 0.46153846153846156,
  'nRequest': 5.5,
  'politeness': 0.9827027196721543,
  'subjectivity': 4.384615384615385,
  'nWords': 166.5,
  'nSentences': 6.5},
 {'sentPos': 1.8,
  'sentNeu': 0.8,
  'sentNeg': 2.2,
  'discourseComp': 0.9583333333333334,
  'plan'